In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

X = np.load("../data/gold/X_train_esm2.npy")
y = np.load("../data/gold/y_train_labels.npy")
device = torch.device("cuda")

dataset = TensorDataset(torch.FloatTensor(X), torch.FloatTensor(y))
loader = DataLoader(dataset, batch_size=256, shuffle=True)

class ProteinClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(320, 512), nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(512, 1024), nn.BatchNorm1d(1024), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(1024, 1500)
        )
    def forward(self, x): return self.net(x)

model = ProteinClassifier().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

print("🏋️ Trenowanie sieci...")
for epoch in range(20):
    model.train()
    total_loss = 0
    for bx, by in loader:
        bx, by = bx.to(device), by.to(device)
        optimizer.zero_grad()
        loss = criterion(model(bx), by)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")

torch.save(model.state_dict(), "../models/protein_nn.pth")

# Generowanie predykcji dla testu
X_test = np.load("../data/gold/X_test_esm2.npy")
model.eval()
with torch.no_grad():
    y_pred_nn = torch.sigmoid(model(torch.FloatTensor(X_test).to(device))).cpu().numpy()
np.save("../data/gold/y_pred_nn.npy", y_pred_nn)
print("✅ Sieć wytrenowana, predykcje zapisane.")